# Spark 101

### 1. Create a spark data frame that contains your favorite programming languages.
 - The name of the column should be language

 - View the schema of the dataframe

 - Output the shape of the dataframe

 - Show the first 5 records in the dataframe


### 2. Load the mpg dataset as a spark dataframe.

 - Create 1 column of output that contains a message like the one below:
     
     - The 1999 audi a4 has a 4 cylinder engine
 
 
 - Transform the trans column so that it only contains either manual or auto
 

### 3. Load the tips dataset as a spark dataframe

   - What percentage of observations are smokers?
    
   - Create a column that contains the tip percentage
    
   - Calculate the average tip percentage for each combination of sex and smoker.
    

### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

   - Convert the temperatures to fahrenheit.
    
   - Which month has the most rain, on average?
    
   - Which year was the windiest?
    
   - What is the most frequent type of weather in January?
    
   - What is the average high and low temperature on sunny days in July in 2013 and 2014?
    
   - What percentage of days were rainy in q3 of 2015?
    
   - For each year, find what percentage of days it rained (had non-zero precipitation)

In [1]:
# imports
import pandas as pd
import numpy as np

import pyspark
import pyspark.sql.functions as F

import multiprocessing

from pyspark.sql.functions import min, max, mean, lit, concat
from pyspark.sql.functions import when

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
# Create a spark data frame that contains your favorite programming languages
df = ['SQL', 'Python', 'Pandas', 'HTML', 'NLP', 'Spark']

In [4]:
# The name of the column should be language
panda_df = pd.DataFrame({'language': df})

In [5]:
# convert pandas df to spark df
df = spark.createDataFrame(panda_df)

In [6]:
# Show the first 5 records in the dataframe
df.show(5)

+--------+
|language|
+--------+
|     SQL|
|  Python|
|  Pandas|
|    HTML|
|     NLP|
+--------+
only showing top 5 rows



In [7]:
# View the schema of the dataframe
df.printSchema()

root
 |-- language: string (nullable = true)



# Load the mpg dataset as a spark dataframe

In [8]:
# imports
from pydataset import data

In [9]:
# getting the mpg data
mpg = spark.createDataFrame(data('mpg'))

In [10]:
# showing the mpg data
mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [11]:
# Create 1 column of output that contains a message like the one below:
# The 1999 audi a4 has a 4 cylinder engine
mpg.select(concat(lit('The '), mpg.year, lit(' '), mpg.manufacturer, lit(' '), mpg.model, lit(' has a'), mpg.cyl, lit('cylinder engine.')).alias('string')).show(truncate=False)

+------------------------------------------------------------+
|string                                                      |
+------------------------------------------------------------+
|The 1999 audi a4 has a4cylinder engine.                     |
|The 1999 audi a4 has a4cylinder engine.                     |
|The 2008 audi a4 has a4cylinder engine.                     |
|The 2008 audi a4 has a4cylinder engine.                     |
|The 1999 audi a4 has a6cylinder engine.                     |
|The 1999 audi a4 has a6cylinder engine.                     |
|The 2008 audi a4 has a6cylinder engine.                     |
|The 1999 audi a4 quattro has a4cylinder engine.             |
|The 1999 audi a4 quattro has a4cylinder engine.             |
|The 2008 audi a4 quattro has a4cylinder engine.             |
|The 2008 audi a4 quattro has a4cylinder engine.             |
|The 1999 audi a4 quattro has a6cylinder engine.             |
|The 1999 audi a4 quattro has a6cylinder engine.       

In [12]:
# Transform the trans column so that it only contains either manual or auto
mpg.select(mpg.trans,
          when((mpg.trans.contains('auto')), 'auto')
          .otherwise('manual')
          .alias('trans')).show(20)

+----------+------+
|     trans| trans|
+----------+------+
|  auto(l5)|  auto|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|  auto|
|  auto(l5)|  auto|
|manual(m5)|manual|
|  auto(av)|  auto|
|manual(m5)|manual|
|  auto(l5)|  auto|
|manual(m6)|manual|
|  auto(s6)|  auto|
|  auto(l5)|  auto|
|manual(m5)|manual|
|  auto(s6)|  auto|
|manual(m6)|manual|
|  auto(l5)|  auto|
|  auto(s6)|  auto|
|  auto(s6)|  auto|
|  auto(l4)|  auto|
|  auto(l4)|  auto|
+----------+------+
only showing top 20 rows



In [13]:
# load the tips dataset as a spark dataframe
tips = spark.createDataFrame(data('tips'))

In [14]:
tips.show(10)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
+----------+----+------+------+---+------+----+
only showing top 10 rows



In [15]:
# What percentage of observations are smokers?
print((tips.filter(tips.smoker == 'Yes').count() / tips.count() * 100), 'percent of observations are smokers.')

38.114754098360656 percent of observations are smokers.


In [16]:
# Create a column that contains the tip percentage

tips = tips.select(
        tips.total_bill, tips.tip, tips.sex, tips.smoker, tips.day, tips.time, tips.size,
        (tips.total_bill / tips.tip).alias('tip_percent')
)

In [17]:
tips.show(10)

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|       tip_percent|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2| 16.82178217821782|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3| 6.228915662650603|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3| 6.002857142857144|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|7.1540785498489425|
|     24.59|3.61|Female|    No|Sun|Dinner|   4| 6.811634349030471|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 5.369426751592356|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|             4.385|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4| 8.615384615384615|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2| 7.673469387755102|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2| 4.575851393188854|
+----------+----+------+------+---+------+----+------------------+
only showing top 10 rows

